In [1]:
import pandas as pd
import numpy as np
train_identity = pd.read_csv('Data/train_identity.csv')
train_transaction = pd.read_csv('Data/train_transaction.csv')

In [2]:
#Storing variable names
identity_vars = list(train_identity)
transaction_vars = list(train_transaction)

#Storing id, fraud and separating from explanatory variables
trans_id = train_transaction['TransactionID']
fraud = train_transaction['isFraud']
x_trans = train_transaction.drop(['TransactionID','isFraud'],axis=1)

In [3]:
#Getting dummies from strings
strings = train_transaction.select_dtypes(include='object')
numerics = train_transaction.select_dtypes(exclude='object')
dummies = pd.get_dummies(strings)
x_trans = pd.concat([dummies, numerics],axis=1)
x_trans = x_trans.drop(['TransactionID','isFraud'],axis=1)

In [4]:
#Getting a count of NaNs
transaction_na_count = x_trans.isnull().sum()
transaction_na_prop = x_trans.isnull().sum()/x_trans.shape[0]*100
transaction_na = pd.concat([transaction_na_count, transaction_na_prop], axis=1)
transaction_na.columns = ['Count','Percentage']

In [10]:
#Filling nas with mean
x_trans = x_trans.fillna(x_trans.mean())
corrs = x_trans.corrwith(fraud)
corrs = abs(corrs.sort_values(ascending=False))
print(corrs)

V257           0.262946
V246           0.251838
V244           0.249951
V242           0.247522
V45            0.236688
V201           0.234520
V200           0.227926
V86            0.224530
V87            0.224450
V189           0.220374
V44            0.218669
V188           0.217058
V258           0.203975
V52            0.201111
V51            0.187440
V228           0.184556
V170           0.178601
V40            0.178413
V79            0.173097
V39            0.170565
V94            0.167984
V38            0.167128
V43            0.166514
V33            0.165534
V199           0.164959
V17            0.164800
V18            0.164689
V74            0.164684
V34            0.162660
V81            0.162608
                 ...   
M8_F           0.043108
V75            0.046516
D5             0.046812
V12            0.047279
M6_F           0.048760
D2             0.051839
D4             0.056450
V36            0.058682
D7             0.063238
M7_F           0.063570
V35            0

Just copying stuff from the EDA notebook, nothing new. We'll first train a logistic regression on the entire dataset and get a ballpark figure for the AUC score, then we'll:
1. Properly set up procedures for cross-validation and calculation of the AUC score
2. Train a regluarized (l1) logistic regression
3. Evaluate it using cross-validation

In [26]:
strong_vars = corrs.index.values[:20]
print(strong_vars)

['V257' 'V246' 'V244' 'V242' 'V45' 'V201' 'V200' 'V86' 'V87' 'V189' 'V44'
 'V188' 'V258' 'V52' 'V51' 'V228' 'V170' 'V40' 'V79' 'V39']


In [27]:
x_subset = x_trans[strong_vars]

In [24]:
from sklearn.metrics import roc_auc_score
from sklearn import model_selection as ms
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression

In [28]:
fraud_logit = LogisticRegression(penalty='l1')

fraud_logit_fit = fraud_logit.fit(X=x_subset,y=fraud)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [35]:
fitted = fraud_logit_fit.predict_proba(X=x_subset)
fraud_prob = pd.DataFrame(fitted[:,1])
print(fraud_prob)
print(fraud)

               0
0       0.025441
1       0.021099
2       0.021099
3       0.021099
4       0.024569
5       0.021099
6       0.021099
7       0.021099
8       0.024569
9       0.021099
10      0.025914
11      0.059649
12      0.025132
13      0.021099
14      0.021099
15      0.025441
16      0.024569
17      0.024569
18      0.021099
19      0.021099
20      0.021099
21      0.021099
22      0.025758
23      0.021099
24      0.021099
25      0.061209
26      0.021099
27      0.021099
28      0.025441
29      0.021099
...          ...
590510  0.021099
590511  0.021099
590512  0.021099
590513  0.021099
590514  0.021099
590515  0.104233
590516  0.021099
590517  0.021099
590518  0.021099
590519  0.021099
590520  0.021099
590521  0.001546
590522  0.021099
590523  0.021099
590524  0.021099
590525  0.021099
590526  0.025758
590527  0.034860
590528  0.035840
590529  0.059649
590530  0.021099
590531  0.025758
590532  0.021099
590533  0.021099
590534  0.062910
590535  0.025441
590536  0.0210

In [36]:
print(roc_auc_score(fraud,fraud_prob))

0.7066573019456849


Initial estimates of the AUC score, not that great considering the Kaggle leaderboard have scores of 0.94+. This is probably higher than what the actual score will be given that I didn't do any cross-validation. Now I'm going to evaluate it using cross-validation to get a proper feel for our out of sample AUC score.

In [41]:
cv_preds = ms.cross_val_predict(fraud_logit, X=x_sub, y=fraud, cv=3,method='predict_proba')

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [49]:
cv_score = roc_auc_score(fraud,cv_preds[:,1])
print(cv_score)

0.707332491179303


Ok, seems like it generalises reasonably well. Note that using cross_val_predict isn't the recommended way to evaluate generalisation error (at least according to the documentation), I'm doing it this way because cross_val_score does not allow me to use predict_proba as the predicted values but uses the predict() method and returns either 0 or 1. This is not desirable as the competition metric is the roc_auc_score based on the predicted probabilities, not the \[0,1\] classification.

Next, we'll try playing around with the number of variables included in the explanatory variable set. I cut it down to 20 variables based on their absolute correlation with the dependent variable. Let's see how increasing/decreasing it has an effect on the cross-validated ROC/AUC score

In [52]:
var_count = np.arange(5,50,5)
cv_scores = []
for var in var_count:
    strong_vars = corrs.index.values[:var]
    x_subset = x_trans[strong_vars]
    cv_preds = ms.cross_val_predict(fraud_logit, X=x_subset, y=fraud, cv=3,method='predict_proba')
    cv_score = roc_auc_score(fraud,cv_preds[:,1])
    cv_scores.append(cv_score)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [53]:
print(cv_scores)

[0.6195014996223323, 0.6339834206517773, 0.7092014132509628, 0.7073443155226837, 0.7143809459979674, 0.7091613350570647, 0.7074994620055817, 0.7070710230126965, 0.707087321610978]
